In [1]:

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from typing import TypedDict

from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel
from langgraph.graph.message import add_messages
from typing import Optional

from dotenv import load_dotenv
import openai
import os
from langchain_core.tools import tool
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import START, END, StateGraph
import pandas as pd
from langchain.tools import tool
from langgraph.prebuilt import ToolNode
import requests
import sys
# Añade la carpeta src al path
sys.path.append(r"C:\Users\David\Documents\Master-Big-Data-Data-Sciencee-e-Inteligencia-Artificial\TFM\AeroGPT\src")

# Importa la función desde el archivo Predictor_RUL.py
from Predictor_RUL import predict_RUL
# Carga las variables de entorno desde el archivo .env
load_dotenv(dotenv_path=".env", override=True)

# Recupera la clave API de OpenAI desde las variables de entorno
openai.api_key = os.getenv("OPENAI_API_KEY")

# Verifica que la clave API se haya cargado correctamente
if openai.api_key is None:
    raise ValueError("La clave API de OpenAI no está configurada correctamente.")

# Ahora puedes usar la clave API en LangChain o directamente con OpenAI
llm_decisions = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini"   # soporta tool-calls modernas
)


c:\Users\David\anaconda3\envs\ml_python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


STATE

In [2]:
class AgentState(BaseModel):
    messages: Annotated[list[BaseMessage], add_messages]
    decision: Optional[Literal['extract', 'none']] = None


NODES

In [3]:
from langchain_core.prompts import ChatPromptTemplate

# Generar el prompt del asistente conversacional
PROMPT_SUPERVISOR = ChatPromptTemplate.from_template(
    """
Eres un supervisor cuyo objetivo es decidir si un mensaje del usuario está relacionado con:

*Extracción de datos sobre motores aeronáuticos (CMAPSS)*  
*Predicción de RUL*  
*Sensores de motores turbofan*  
*Datos de ciclos, settings, condiciones operativas, o sensores FI, HPC, LPT, etc.*

Reglas:
- Si el mensaje del usuario **está relacionado con CMAPSS, motores, sensores o RUL**, responde **solo** con:
    extract
- Si el mensaje **NO está relacionado**, responde **solo** con:
    none
- No agregues explicaciones, frases adicionales ni símbolos.

Mensaje del usuario: {user_message}

Tu respuesta: (extract/none)
"""
)


PROMTP_EXTRACT_CMAPSS = ChatPromptTemplate.from_template(
    """
   Eres un asistente especializado en extraer datos estructurados para alimentar un modelo de predicción RUL basado en CMAPSS.

   TU TAREA:
   Extraer únicamente la información explícita mencionada por el usuario sobre el estado actual de un motor aeronáutico.

   NO DEBES inventar valores.  
   NO estimes sensores no mencionados.  
   NO rellenes medias ni interpolaciones: eso lo hará el modelo después.

   ------------------------------------------------------------
   DATOS QUE DEBES EXTRAER
   ------------------------------------------------------------

   1. unidad  
      - Identificador del motor (si no se menciona → 000)

   2. tiempo_ciclos  
      - Ciclo operativo actual (si no se menciona → 000)

   3. configuraciones_operativas  
      - Tres valores: setting_1, setting_2, setting_3  
      - Si el usuario no menciona alguno → 000

   4. mediciones_sensores  
      - Lista EXACTA de 21 sensores (s_1 a s_21)  
      - Si el usuario menciona un sensor (“sensor 7: 550”) asigna ese valor.  
      - Si NO lo menciona → 000.

   IMPORTANTE:
   - NO inventes datos.
   - NO rellenes con medias.
   - NO derives valores no mencionados.

   ------------------------------------------------------------
   SELECCIÓN DEL MODELO (FD)
   ------------------------------------------------------------

   Selecciona el modelo usando estas reglas:

   - FD001 → condiciones de nivel del mar + solo HPC degradation
   - FD002 → condiciones SEIS + solo HPC
   - FD003 → nivel del mar + HPC y/o Fan degradation
   - FD004 → condiciones SEIS + HPC y/o Fan degradation

   Reglas de selección:
   - Si se menciona “nivel del mar”, “sea level” → FD001 o FD003
   - Si se mencionan múltiples condiciones, ambiente variable, altitud variable → FD002 o FD004
   - Si se menciona “fan”, “fan degradation”, “fan speed issues” → usar FD003 o FD004
   - Si solo se menciona HPC degradation → usar FD001 o FD002
   - Si no hay contexto → seleccionar FD001

   ------------------------------------------------------------
   FORMATO DE RESPUESTA (estricto JSON)
   ------------------------------------------------------------

   {{
   "unidad": <int|000>,
   "tiempo_ciclos": <int|000>,
   "configuraciones_operativas": [setting_1, setting_2, setting_3],
   "mediciones_sensores": {{
         "s_1": <float|000>,
         "s_2": <float|000>,
         ...
         "s_21": <float|000>
   }},
   "modelo_seleccionado": "FD001" | "FD002" | "FD003" | "FD004"
   }}

   ------------------------------------------------------------
   MENSAJE DEL USUARIO:
   ------------------------------------------------------------
   {message}
   """
)

TOOLS

In [4]:
@tool
def extract_cmapss_tool(message: str) -> str:
    """
    Usa esta herramienta SOLO cuando el supervisor decida 'extract'.
    Extrae datos CMAPSS desde un texto del usuario.
    """
    chain = PROMTP_EXTRACT_CMAPSS | llm_decisions
    response = chain.invoke({"message": message})

    # Limpiar output para que sea JSON válido
    tool_output = response.content
    # Quitar backticks si existen
    tool_output = tool_output.replace("```json", "").replace("```", "")
    # Reemplazar 000 por 0
    tool_output = tool_output.replace(": 000", ": 0")

    print(f"TOOL: {tool_output}")
    return tool_output


@tool
def tool_output_to_df(tool_output: dict) -> pd.DataFrame:
    """Extrae configuraciones CMAPSS y las convierte en diccionario plano"""
    settings = tool_output.get("configuraciones_operativas", [0,0,0])
    sensor_vals = tool_output.get("mediciones_sensores", {f"s_{i}": 0 for i in range(1,22)})

    # Creamos diccionario plano con nombres que espera el modelo
    data = {
        "setting_1": [settings[0]],
        "setting_2": [settings[1]],
        "setting_3": [settings[2]],
        **{k: [v] for k, v in sensor_vals.items()}
    }

    return pd.DataFrame(data)

In [5]:
def supervisor_action(state: AgentState):
    """Decide si una pregunta del usuario es una consulta de CMAPSS o si no tiene que ver con esto."""
    
    
    chain = PROMPT_SUPERVISOR | llm_decisions
    response = chain.invoke({"user_message": state.messages[-1].content})
    
    decision = response.content.strip()
    
    print(f">>> Supervisor: {decision}")
    return {'decision': decision}

def extract_cmapss_action(state: AgentState):
    """Extrae datos CMAPSS y predice RUL usando el modelo GRU."""
    last_user_msg = state.messages[-1].content

    # Ejecuta la tool
    tool_result_str = extract_cmapss_tool.run(last_user_msg)
    tool_result = json.loads(tool_result_str)

    # Convertir a DataFrame pasando dict en el campo esperado
    df_user = tool_output_to_df.run({"tool_output": tool_result})

    # Predicción RUL
    base_path = r"C:\Users\David\Documents\Master-Big-Data-Data-Sciencee-e-Inteligencia-Artificial\TFM\AeroGPT\data\CMAPSS"
    rul_pred = predict_RUL(df_user, base_path, fd=tool_result["modelo_seleccionado"])

    print(f">>> RUL predicho: {rul_pred['predicted_RUL']}")

    return {"messages": [AIMessage(content=json.dumps(rul_pred))]}


    

Routine Logic

In [6]:
def supervisor_decision(state: AgentState):
    if state.decision == 'extract':
        return "Extractor"
    else:
        return END
    
# def extract_tool_decision(state: AgentState):
#     """Si ya se ejecutó la tool, termina."""
#     ai_msg = state.messages[-1]
#     if hasattr(ai_msg, "tool_calls") and ai_msg.tool_calls:
#         return END
#     return END

def extract_tool_decision(state: AgentState):
    """Después de ejecutar la tool, termina el flujo."""
    return END




In [7]:
def update_graph():
    graph = StateGraph(AgentState)

    graph.add_node("Supervisor", supervisor_action)
    graph.add_node("Extractor", extract_cmapss_action)

    graph.add_edge(START, "Supervisor")
    graph.add_conditional_edges("Supervisor", supervisor_decision)
    graph.add_conditional_edges("Extractor", extract_tool_decision)

    return graph.compile()


In [8]:
graph = update_graph()

user_input = input("\nPregunta del usuario: ('stop' para parar)")
while user_input != 'stop':
    messages = graph.invoke(
        input={"messages": [HumanMessage(content=f"{user_input}")]}
    )
    user_input = input("\nPregunta del usuario: ('stop' para parar)")


>>> Supervisor: extract
TOOL: 
{
   "unidad": 1,
   "tiempo_ciclos": 0,
   "configuraciones_operativas": [2, 3, 4],
   "mediciones_sensores": {
         "s_1": 100,
         "s_2": 100,
         "s_3": 100,
         "s_4": 100,
         "s_5": 100,
         "s_6": 100,
         "s_7": 100,
         "s_8": 100,
         "s_9": 100,
         "s_10": 100,
         "s_11": 100,
         "s_12": 100,
         "s_13": 100,
         "s_14": 100,
         "s_15": 100,
         "s_16": 100,
         "s_17": 100,
         "s_18": 100,
         "s_19": 100,
         "s_20": 100,
         "s_21": 100
   },
   "modelo_seleccionado": "FD001"
}



C:\Users\David\Documents\Master-Big-Data-Data-Sciencee-e-Inteligencia-Artificial\TFM\AeroGPT\src\Predictor_RUL.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.loa

>>> RUL predicho: 141.95713806152344


c:\Users\David\anaconda3\envs\ml_python\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


>>> Supervisor: extract
TOOL: 
{
   "unidad": 1,
   "tiempo_ciclos": 0,
   "configuraciones_operativas": [2, 3, 4],
   "mediciones_sensores": {
         "s_1": 100,
         "s_2": 100,
         "s_3": 100,
         "s_4": 100,
         "s_5": 100,
         "s_6": 100,
         "s_7": 100,
         "s_8": 100,
         "s_9": 100,
         "s_10": 100,
         "s_11": 100,
         "s_12": 100,
         "s_13": 100,
         "s_14": 100,
         "s_15": 100,
         "s_16": 100,
         "s_17": 100,
         "s_18": 100,
         "s_19": 100,
         "s_20": 100,
         "s_21": 100
   },
   "modelo_seleccionado": "FD004"
}

>>> RUL predicho: 65.17757415771484


C:\Users\David\Documents\Master-Big-Data-Data-Sciencee-e-Inteligencia-Artificial\TFM\AeroGPT\src\Predictor_RUL.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.loa